In [9]:
import mytorch.nn as nn
import torch

In [2]:
x = nn.Linear(1,128)
e = nn.Embedding(27,10)

In [4]:
x.training, e.training

(True, True)

In [5]:
x.bias, e.weight

(tensor([ 0.0276, -0.1089, -0.2495,  0.1641,  0.1384,  0.0119, -0.0211,  0.0704,
          0.0748,  0.1393,  0.1491, -0.0707,  0.0580,  0.0153,  0.0451, -0.0125,
          0.0884, -0.0047, -0.0794, -0.0790, -0.1147, -0.1840, -0.1145, -0.1203,
          0.0192,  0.0360, -0.0536, -0.0617,  0.0585, -0.1100,  0.2305,  0.0928,
          0.0683, -0.0542, -0.1283, -0.1202,  0.0696, -0.0080, -0.1876, -0.0278,
          0.0434,  0.0412,  0.0423,  0.0271,  0.1463,  0.0730,  0.0278, -0.1393,
          0.0732, -0.2225, -0.0486, -0.0352, -0.0320, -0.0136, -0.0197,  0.1290,
          0.1130, -0.1045,  0.2131,  0.1154,  0.0984, -0.0156,  0.0441, -0.0707,
          0.0347,  0.0104, -0.0618,  0.1383,  0.1717, -0.0763,  0.0907,  0.1616,
         -0.0077, -0.1232,  0.0536,  0.2006, -0.2180, -0.0227,  0.1911,  0.0135,
          0.1277, -0.0810, -0.0222, -0.0414,  0.0169,  0.1283, -0.0616, -0.2072,
         -0.0370,  0.1877, -0.0080,  0.0209, -0.0242,  0.0609, -0.0396,  0.0642,
         -0.1154, -0.1917,  

In [6]:
print (x)
print (e)

Linear(in_features=1, out_features=128, bias=True), Total parameters: 256
Embedding(num_embeddings=27, embedding_dim=10), Total parameters: 270


In [13]:
IX = torch.ones((2,2)).long()

In [20]:
print (IX.shape)
print (e.weight.shape)
(e(IX)).shape

torch.Size([2, 2])
torch.Size([27, 10])


torch.Size([2, 2, 10])